In [69]:
import pandas as pd
import numpy as np


def filter_dataframes(dataframes, query_string):
    """
    Filter a list of DataFrames using a query string.

    Parameters:
        dataframes (dict): Dictionary of pandas DataFrames.
        query_string (str): Query string to filter the DataFrames.

    Returns:
        dict: Dictionary of filtered DataFrames.
    """
    filtered_dataframes = {key: df.query(query_string).copy() for key, df in dataframes.items()}
    return filtered_dataframes


def calculate_sum_columns(dataframes_dict, columns):
    """
    Calculate the sum of specified columns for each DataFrame in a dictionary.

    Parameters:
        dataframes_dict (dict): Dictionary of pandas DataFrames.
        columns (list): List of column names for which sum needs to be calculated.

    Returns:
        pandas DataFrame: DataFrame containing the sums of specified columns for each DataFrame,
                          with DataFrame keys as index.
    """
    # Dictionary to store sums for each DataFrame
    sums = {}

    # Calculate sum for each DataFrame
    for key, df in dataframes_dict.items():
        # Select specified columns and calculate sum
        sums[key] = df[columns].sum()

    # Concatenate sums into a DataFrame
    result_df = pd.concat(sums, axis=1).T

    return result_df


def iterate_queries(dataframes, query_strings, columns_to_sum):
    """
    Iterate over a dictionary of query strings, filter DataFrames, and calculate the sum of 
    specified columns.

    Parameters:
        dataframes (dict): Dictionary of pandas DataFrames.
        query_strings (dict): Dictionary of query strings.
        columns_to_sum (list): List of column names for which sum needs to be calculated.

    Returns:
        dict: Dictionary containing filtered DataFrames and their sums for each query string.
    """
    result = {}

    # Iterate over each query string
    for query_name, query_string in query_strings.items():
        # Filter DataFrames
        filtered_dfs = filter_dataframes(dataframes, query_string)

        # Calculate sum of columns for filtered DataFrames
        sums_dfs = calculate_sum_columns(filtered_dfs, columns_to_sum)

        # Store filtered DataFrames and their sums
        result[query_name] = sums_dfs

    return result


ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [70]:
df = pd.read_excel(
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
).iloc[:-2]

df = df.rename(columns={'Sexo\n1=Hombre\n2=Mujer': "hombre_mujer"})
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

# Cálculo de Personas por Estrato

Aquí se quiere calcular la cantidad de personas que hay en los siguientes estratos:

1. Hombres y Mujeres
2. Hombres
3. Mujeres
4. Población sobre 45 años
5. Población sobre 60 años
6. Población entre 60 a 70 años
7. Población recién nacidos
8. Población sobre 65 años

Para cada estrato se calculará la población que hay para el SSMO, Región Metropolitana y el País.

In [71]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

In [72]:
result_df = calculate_sum_columns(ESTRATOS_A_CALCULAR, COLUMNAS_POBLACION)

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [73]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
}

In [74]:
results = iterate_queries(ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION)